In addition to the libraries used in the shorter example, this script also requires PyQt4. On the command line, type `pip install PyQt4` to install. The main difference between this script and the example script are: 

1. There are many more nested try-except pairs to account for the duplicate Tweet issue.
1. It scrapes more sites. 
1. Not all of the sites scraped output nice HTML. So this has to be addressed. FiveThirtyEight, for example, returns compressed content. So gzip is needed to uncompressed and read the content. Predictwise fills in its data using javascript. So we have to pretend to be a browser and run the javascript before scraping. Hence, the use of PyQt4.

That being said, I haven't annotated this code. That's what the example was for. So if you're here to learn, you'll have to poke around and play.

In [1]:
import datetime
now = datetime.datetime.now()

import re
import urllib.request
import gzip

###

import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 

#Take this class for granted.Just use result of rendering.
class Render(QWebPage):  
  def __init__(self, url):  
    self.app = QApplication(sys.argv)  
    QWebPage.__init__(self)  
    self.loadFinished.connect(self._loadFinished)  
    self.mainFrame().load(QUrl(url))  
    self.app.exec_()  
  
  def _loadFinished(self, result):  
    self.frame = self.mainFrame()  
    self.app.quit()  

In [2]:
url_6 = "http://predictwise.com/politics/2016-president-winner" 
r = Render(url_6)  
p_6 = r.frame.toHtml()
res_6 = re.search(r'Democratic</div><div class="list-value">(\d*)',p_6)

In [3]:
url_1 = "http://www.nytimes.com/interactive/2016/upshot/presidential-polls-forecast.html"
p_1 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_1).read()
#print(p_1)
res_1 = re.search(b'<strong class="g-leader-pct">(.*)\%',p_1)

url_2 = "http://projects.fivethirtyeight.com/2016-election-forecast/"
p_2 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_2)
p_2 = gzip.GzipFile(fileobj=p_2).read() 
#print(p_2)
res_2 = re.search(b'data-party="D" class="candidate-val winprob">(.{4})',p_2)

url_3 = "http://election.princeton.edu/2012/09/29/the-short-term-presidential-predictor/"
p_3 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_3).read()
#print(p_3)
res_3 = re.search(b'/the-short-term-presidential-predictor/">Bayesian\s*(.*)\%',p_3)

url_4 = "http://elections.dailykos.com/app/data/assign_forecast_history_2016.js"
p_4 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_4).read()
#print(p_4)
res_4 = re.search(b'(\d\.\d*)}\],"senator":\[.*$',p_4)    

url_5 = "https://www.betfairpredicts.com/politics"
p_5 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_5).read()
#print(p_5)
res_5 = re.search(b'107373419", "percent_value": (\d*\.\d*), "slug": "hillary',p_5)   

import twitter
from twitter import TwitterError
api = twitter.Api(consumer_key='[YOUR KEY]',
                  consumer_secret='[YOUR SECRET]',
                  access_token_key='[TOKEN KEY]',
                  access_token_secret='[TOKEN SECRET]')

fired = 0

if res_1 and res_2 and res_3 and res_4 and res_5 and res_6:
    output_1 = round(float(res_1.group(1).decode('UTF-8')))
    output_2 = float(res_2.group(1).decode('UTF-8'))
    output_3 = round(float(res_3.group(1).decode('UTF-8')))
    output_4 = float(res_4.group(1).decode('UTF-8'))
    output_4  = round(output_4*100)
    output_5 = float(res_5.group(1).decode('UTF-8'))
    output_5  = round(output_5)
    output_6 = float(res_6.group(1))
    output_6  = round(output_6)
    average = round((float(output_1) + float(output_2) + float(output_3) + float(output_4) + float(output_5) + float(output_6))/6)
    document_key = "[YOUR DOCUMENT ID/KEY]"
    import csv
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    scope = ['https://spreadsheets.google.com/feeds']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('[LOCATION OF JSON FILE]', scope)
    gc = gspread.authorize(credentials)
    wks = gc.open_by_key(document_key)
    worksheet = wks.worksheet("Data")
    worksheet.resize(worksheet.row_count)
    print("%s | %s, %s, %s, %s, %s, %s | %s"%(worksheet.row_values(worksheet.row_count)[7],worksheet.row_values(worksheet.row_count)[1],worksheet.row_values(worksheet.row_count)[2],worksheet.row_values(worksheet.row_count)[3],worksheet.row_values(worksheet.row_count)[4],worksheet.row_values(worksheet.row_count)[5],worksheet.row_values(worksheet.row_count)[6],worksheet.row_values(worksheet.row_count)[0]))
    print("%s | %s, %s, %s, %s, %s, %s | %s"%(average,output_1,output_2,output_3,output_4,output_5,output_6,now))
    if (float(worksheet.row_values(worksheet.row_count)[1]) != output_1) or (float(worksheet.row_values(worksheet.row_count)[2]) != output_2) or (float(worksheet.row_values(worksheet.row_count)[3]) != output_3) or (float(worksheet.row_values(worksheet.row_count)[4]) != output_4) or (float(worksheet.row_values(worksheet.row_count)[5]) != output_5) or (float(worksheet.row_values(worksheet.row_count)[6]) != output_6):

        if (float(worksheet.row_values(worksheet.row_count)[1]) != output_1):
            fired = 1
            if float(worksheet.row_values(worksheet.row_count)[1]) > output_1: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("1.1")
                status = api.PostUpdate('.@UpshotNYT puts Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_1,direction,worksheet.row_values(worksheet.row_count)[1],url_1))
                print(status.text)
            except TwitterError:
                print("1.2")
                status = api.PostUpdate('.@UpshotNYT pegs Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_1,direction,worksheet.row_values(worksheet.row_count)[1],url_1))
                print(status.text)

        if (float(worksheet.row_values(worksheet.row_count)[2]) != output_2):
            fired = 1
            if float(worksheet.row_values(worksheet.row_count)[2]) > output_2: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("2.1")
                status = api.PostUpdate('.@FiveThirtyEight puts Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_2,direction,worksheet.row_values(worksheet.row_count)[2],url_2))
                print(status.text)
            except TwitterError:
                print("2.2")
                status = api.PostUpdate('.@FiveThirtyEight pegs Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_2,direction,worksheet.row_values(worksheet.row_count)[2],url_2))
                print(status.text)

        if (float(worksheet.row_values(worksheet.row_count)[3]) != output_3):
            fired = 1
            if float(worksheet.row_values(worksheet.row_count)[3]) > output_3: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("3.1")
                status = api.PostUpdate('.@Princeton Election Consortium puts Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_3,direction,worksheet.row_values(worksheet.row_count)[3],url_3))
                print(status.text)
            except TwitterError:
                print("3.2")
                status = api.PostUpdate('.@Princeton Election Consortium pegs Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_3,direction,worksheet.row_values(worksheet.row_count)[3],url_3))
                print(status.text)

        if (float(worksheet.row_values(worksheet.row_count)[4]) != output_4):
            fired = 1
            if float(worksheet.row_values(worksheet.row_count)[4]) > output_4: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("4.1")
                status = api.PostUpdate('.@DailyKos puts Clinton\'s chance of winning at %s%% (%s from %s%%). http://elections.dailykos.com/app/elections/2016'%(output_4,direction,worksheet.row_values(worksheet.row_count)[4]))
                print(status.text)
            except TwitterError:
                print("4.2")
                status = api.PostUpdate('.@DailyKos pegs Clinton\'s chance of winning at %s%% (%s from %s%%). http://elections.dailykos.com/app/elections/2016'%(output_4,direction,worksheet.row_values(worksheet.row_count)[4]))
                print(status.text)

        if (float(worksheet.row_values(worksheet.row_count)[6]) != output_6):
            fired = 1
            if float(worksheet.row_values(worksheet.row_count)[6]) > output_6: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("6.1")
                status = api.PostUpdate('.@PredictWise puts Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_6,direction,worksheet.row_values(worksheet.row_count)[6],url_6))
                print(status.text)
            except TwitterError:
                print("6.2")
                status = api.PostUpdate('.@PredictWise pegs Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_6,direction,worksheet.row_values(worksheet.row_count)[6],url_6))
                print(status.text)

        if ((float(worksheet.row_values(worksheet.row_count)[5]) != output_5) and ((float(worksheet.row_values(worksheet.row_count)[7]) != average) or (fired == 1))):
            fired = 1
            if float(worksheet.row_values(worksheet.row_count)[5]) > output_5: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("5.1")
                status = api.PostUpdate('.@BetfairUSA puts Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_5,direction,worksheet.row_values(worksheet.row_count)[5],url_5))
                print(status.text)
            except TwitterError:
                try:
                    print("5.2")
                    status = api.PostUpdate('.@BetfairUSA pegs Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_5,direction,worksheet.row_values(worksheet.row_count)[5],url_5))
                    print(status.text)
                except TwitterError:
                    try:
                        print("5.3")
                        status = api.PostUpdate('.@BetfairUSA places Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_5,direction,worksheet.row_values(worksheet.row_count)[5],url_5))
                        print(status.text)
                    except TwitterError:
                        print("5.4")
                        status = api.PostUpdate('.@BetfairUSA has Clinton\'s chance of winning at %s%% (%s from %s%%). %s'%(output_5,direction,worksheet.row_values(worksheet.row_count)[5],url_5))
                        print(status.text)
        else:
            print("5.5")
            output_5 = float(worksheet.row_values(worksheet.row_count)[5])                
                
        if (float(worksheet.row_values(worksheet.row_count)[7]) != average):
            if float(worksheet.row_values(worksheet.row_count)[7]) > average: 
                direction = "down"
            else:
                direction = "up"
            try:
                print("average.1.1")
                status = api.PostUpdate('The 6-forecast average puts Clinton\'s chance of winning at %s%% (%s from %s%%). http://bit.ly/meanvoter'%(average,direction,worksheet.row_values(worksheet.row_count)[7]))
                print(status.text)
            except TwitterError:
                try:
                    print("average.1.2")
                    status = api.PostUpdate('The 6-forecast average pegs Clinton\'s chance of winning at %s%% (%s from %s%%). http://bit.ly/meanvoter'%(average,direction,worksheet.row_values(worksheet.row_count)[7]))
                    print(status.text)
                except TwitterError:
                    try:
                        print("average.1.3")
                        status = api.PostUpdate('The 6-forecast average places Clinton\'s chance of winning at %s%% (%s from %s%%). http://bit.ly/meanvoter'%(average,direction,worksheet.row_values(worksheet.row_count)[7]))
                        print(status.text)
                    except TwitterError:
                        print("average.1.4")
                        status = api.PostUpdate('The 6-forecast average has Clinton\'s chance of winning at %s%% (%s from %s%%). http://bit.ly/meanvoter'%(average,direction,worksheet.row_values(worksheet.row_count)[7]))
                        print(status.text)
        elif (fired == 1):
            try:
                print("average.2.1")
                status = api.PostUpdate('The 6-forecast average still puts Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                print(status.text)
            except TwitterError:
                try:
                    print("average.2.2")
                    status = api.PostUpdate('The 6-forecast average still places Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                    print(status.text)
                except TwitterError:
                    try:
                        print("average.2.3")
                        status = api.PostUpdate('The 6-forecast average still has Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                        print(status.text)
                    except TwitterError:
                        try:
                            print("average.2.4")
                            status = api.PostUpdate('The 6-forecast average is steady with Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                            print(status.text)
                        except TwitterError:
                                try:
                                    print("average.2.5")
                                    status = api.PostUpdate('The 6-forecast average still pegs Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                                    print(status.text)
                                except TwitterError:
                                        try:
                                            print("average.2.6")
                                            status = api.PostUpdate('The 6-forecast average is still putting Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                                            print(status.text)
                                        except TwitterError:
                                            try:
                                                print("average.2.7")
                                                status = api.PostUpdate('The 6-forecast average is still pegging Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                                                print(status.text)
                                            except TwitterError:
                                                print("average.2.8")
                                                status = api.PostUpdate('The 6-forecast average is still placing Clinton\'s chance of winning at %s%%. http://bit.ly/meanvoter'%(average))
                                                print(status.text)
        if (fired == 1):
            worksheet.append_row([now,output_1,output_2,output_3,output_4,output_5,output_6,average])

75 | 80, 68.7, 90, 72, 66, 74 | 2016-09-13 21:00:04
75 | 80, 68.7, 90, 72, 66, 74 | 2016-09-13 21:49:40.561764
